# Load and Prepare Raw data and Save tables in MongoDB


In [1]:
from typing import List
import requests
import json
import pandas as pd
import numpy as np
import time
import sys
from collections.abc import MutableMapping,Iterable


def get_API_response(url):
    response_API = requests.get(url)
    print('Status Code-', response_API.status_code)
    print('Loading Data--')
    start = time.time()
    raw_data = response_API.text
    # Parse Json
    data = json.loads(raw_data)
    print('Time taken to load data-',round((time.time()-start),3))
    return data

## Basic info of stable coins

In [14]:
# List all stablecoins along with their circulating amounts from defilama
url_SC_basic_info = 'https://stablecoins.llama.fi/stablecoins?includePrices=true'
# ?includePrices=true
raw_SC_basic_info = get_API_response(url_SC_basic_info)
raw_SC_basic_info_list =raw_SC_basic_info['peggedAssets']
pdf_SC_basic_info = pd.json_normalize(raw_SC_basic_info_list, sep='_')


Status Code- 200
Loading Data--
Time taken to load data- 0.0


In [18]:
select_keys = ['id', 'name', 'symbol', 'gecko_id', 'pegType', 'priceSource', 'circulating_peggedUSD', 'chains', 'price']
pdf_SC_basic_info_2 = pdf_SC_basic_info[select_keys]

In [30]:
pdf_SC_basic_info_2.head(10)

,id,name,symbol,gecko_id,pegType,priceSource,circulating_peggedUSD,chains,price
0,1,Tether,USDT,tether,peggedUSD,chainlink,6.633972e+10,"[Tron, Ethereum, Solana, Omni, BSC, Avalanche,...",1.000009
1,2,USD Coin,USDC,usd-coin,peggedUSD,chainlink,5.404817e+10,"[Ethereum, Solana, Tron, Avalanche, BSC, Polyg...",0.999900
2,3,TerraClassicUSD,USTC,terrausd,peggedUSD,chainlink,1.127878e+10,"[Terra Classic, Ethereum, BSC, Avalanche, Poly...",0.043316
3,4,Binance USD,BUSD,binance-usd,peggedUSD,chainlink,1.809741e+10,"[Ethereum, BSC, Kava, Astar, Harmony, Solana, ...",0.999900
4,5,Dai,DAI,dai,peggedUSD,chainlink,6.787235e+09,"[Ethereum, Avalanche, BSC, Fantom, Polygon, Gn...",0.999869
5,6,Frax,FRAX,frax,peggedUSD,chainlink,1.366059e+09,"[Ethereum, Fantom, Moonriver, Arbitrum, Polygo...",0.999938
6,7,TrueUSD,TUSD,true-usd,peggedUSD,chainlink,1.248782e+09,"[Ethereum, Tron, BSC, Cronos, Polygon, Avalanc...",0.999817
7,8,Liquity USD,LUSD,liquity-usd,peggedUSD,chainlink,1.739994e+08,"[Ethereum, Optimism, Polygon, Avalanche, BSC, ...",1.027756
8,9,Fei USD,FEI,fei-usd,peggedUSD,chainlink,2.010146e+08,[Ethereum],0.999199
9,10,Magic Internet Money,MIM,magic-internet-money,peggedUSD,chainlink,2.118703e+08,"[Ethereum, Avalanche, Fantom, Arbitrum, Metis,...",0.998702


## Historical MCap of SCs

In [39]:
# Get historical mcap sum of all stablecoins in all chain
stablecoin_id = 1
url_SC_Mcap_allChains = 'https://stablecoins.llama.fi/stablecoincharts/all?stablecoin='+str(stablecoin_id)
raw_SC_Mcap_allChainst = get_API_response(url_SC_Mcap_allChains)
raw_SC_Mcap_allChainst_list =raw_SC_Mcap_allChainst
pdf_SC_Mcap_allChainst= pd.json_normalize(raw_SC_Mcap_allChainst_list, sep='_')
pdf_SC_Mcap_allChainst['date'] = pd.to_datetime(pdf_SC_Mcap_allChainst['date'] , unit='s')

Status Code- 200
Loading Data--
Time taken to load data- 0.008


In [40]:
pdf_SC_Mcap_allChainst

,date,totalCirculating_peggedUSD,totalUnreleased_peggedUSD,totalCirculatingUSD_peggedUSD,totalMintedUSD_peggedUSD,totalBridgedToUSD_peggedUSD
0,2020-01-03,4.288899e+09,0.000000e+00,4.288899e+09,0,0
1,2020-01-04,4.283023e+09,0.000000e+00,4.283023e+09,0,0
2,2020-01-05,4.287221e+09,0.000000e+00,4.287221e+09,0,0
3,2020-01-06,4.281392e+09,0.000000e+00,4.281392e+09,0,0
4,2020-01-07,4.285344e+09,0.000000e+00,4.285344e+09,0,0
...,...,...,...,...,...,...
924,2022-07-15,6.634976e+10,2.258729e+09,6.631793e+10,0,0
925,2022-07-16,6.634976e+10,2.258729e+09,6.631793e+10,0,0
926,2022-07-17,6.634976e+10,2.258729e+09,6.634945e+10,0,0
927,2022-07-18,6.636972e+10,2.238769e+09,6.636941e+10,0,0


In [54]:
pdf_SC_basic_info_2['id'][i]

'39'

In [55]:
frax_start_date= '2020-12-21'
list_pdf_SC_Mcap_allChainst =[]
for i in range(pdf_SC_basic_info_2.shape[0]):
    try:
        print(i,pdf_SC_basic_info_2['name'][i], pdf_SC_basic_info_2['symbol'][i])
        
        stablecoin_id = pdf_SC_basic_info_2['id'][i]
        symbol = pdf_SC_basic_info_2['symbol'][i]

        url_SC_Mcap_allChains = 'https://stablecoins.llama.fi/stablecoincharts/all?stablecoin='+str(stablecoin_id)
        raw_SC_Mcap_allChainst = get_API_response(url_SC_Mcap_allChains)
        
        pdf_SC_Mcap_allChainst= pd.json_normalize(raw_SC_Mcap_allChainst, sep='_')
        pdf_SC_Mcap_allChainst['date'] = pd.to_datetime(pdf_SC_Mcap_allChainst['date'] , unit='s')
        pdf_SC_Mcap_allChainst = pdf_SC_Mcap_allChainst[pdf_SC_Mcap_allChainst['date']>= frax_start_date]
        pdf_SC_Mcap_allChainst['id'] = stablecoin_id
        pdf_SC_Mcap_allChainst['symbol'] = symbol
        pdf_SC_Mcap_allChainst = pdf_SC_Mcap_allChainst[['id','symbol','date','totalCirculating_peggedUSD','totalUnreleased_peggedUSD','totalCirculatingUSD_peggedUSD','totalMintedUSD_peggedUSD','totalBridgedToUSD_peggedUSD']]
        list_pdf_SC_Mcap_allChainst.append(pdf_SC_Mcap_allChainst)
    except Exception as e:
        print('Not present and skipping..')
pdf_SC_Mcap_allChains = pd.DataFrame()
pdf_SC_Mcap_allChains = pd.concat(list_pdf_SC_Mcap_allChainst)
pdf_SC_Mcap_allChains.reset_index(inplace=True)
pdf_SC_Mcap_allChains.drop('index',axis=1, inplace = True)


0 Tether USDT
Status Code- 200
Loading Data--
Time taken to load data- 0.0
1 USD Coin USDC
Status Code- 200
Loading Data--
Time taken to load data- 0.142
2 TerraClassicUSD USTC
Status Code- 200
Loading Data--
Time taken to load data- 0.008
3 Binance USD BUSD
Status Code- 200
Loading Data--
Time taken to load data- 0.009
4 Dai DAI
Status Code- 200
Loading Data--
Time taken to load data- 0.008
5 Frax FRAX
Status Code- 200
Loading Data--
Time taken to load data- 0.008
6 TrueUSD TUSD
Status Code- 200
Loading Data--
Time taken to load data- 0.008
7 Liquity USD LUSD
Status Code- 200
Loading Data--
Time taken to load data- 0.0
8 Fei USD FEI
Status Code- 200
Loading Data--
Time taken to load data- 0.009
9 Magic Internet Money MIM
Status Code- 200
Loading Data--
Time taken to load data- 0.008
10 Pax Dollar USDP
Status Code- 200
Loading Data--
Time taken to load data- 0.008
11 Neutrino USD USDN
Status Code- 200
Loading Data--
Time taken to load data- 0.008
12 YUSD Stablecoin YUSD
Status Code- 20

In [60]:
pdf_SC_Mcap_allChains

,id,symbol,date,totalCirculating_peggedUSD,totalUnreleased_peggedUSD,totalCirculatingUSD_peggedUSD,totalMintedUSD_peggedUSD,totalBridgedToUSD_peggedUSD
0,1,USDT,2020-12-21,2.036340e+10,0.0,2.036340e+10,0,0
1,1,USDT,2020-12-22,2.035257e+10,0.0,2.035257e+10,0,0
2,1,USDT,2020-12-23,2.047374e+10,0.0,2.047374e+10,0,0
3,1,USDT,2020-12-24,2.055903e+10,0.0,2.055903e+10,0,0
4,1,USDT,2020-12-25,2.058810e+10,0.0,2.058810e+10,0,0
...,...,...,...,...,...,...,...,...
14153,54,USN,2022-07-19,1.275757e+08,0.0,1.274431e+08,0,0
14154,57,USH,2022-07-16,5.394679e+06,0.0,5.394679e+06,0,0
14155,57,USH,2022-07-17,5.409085e+06,0.0,5.408660e+06,0,0
14156,57,USH,2022-07-18,5.438402e+06,0.0,5.437975e+06,0,0


## TVL related data

In [85]:
# Get All protocols in defilama along with their tvl
url_tvl_protocols = 'https://api.llama.fi/protocols'
raw_tvl_protocols = get_API_response(url_tvl_protocols) 
raw_tvl_protocols_list = raw_tvl_protocols
pdf_tvl_protocols = pd.json_normalize(raw_tvl_protocols_list, sep='_')


Status Code- 200
Loading Data--
Time taken to load data- 0.032


In [86]:
select_columns = ['id', 'name', 'address', 'symbol','chain', 'gecko_id',\
    'audits', 'cmcId','category', 'chains' ,'oracles', 'slug','tvl','fdv', 'mcap']
    
pdf_tvl_protocols = pdf_tvl_protocols[select_columns]

# All columns
# ['id', 'name', 'address', 'symbol', 'url', 
# 'description', 'chain', 'logo', 'audits', 
# 'audit_note', 'gecko_id', 'cmcId', 'category', 
# 'chains', 'module', 'twitter', 'audit_links', 
# 'oracles', 'slug', 'tvl', 'chainTvls', 
# 'change_1h', 'change_1d', 'change_7d', 'fdv', 'mcap']

In [89]:
pdf_tvl_protocols.head(10)
# 1738 protocols enlisted

,id,name,address,symbol,chain,gecko_id,audits,cmcId,category,chains,oracles,slug,tvl,fdv,mcap
0,118,MakerDAO,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,Ethereum,maker,2,1518,CDP,[Ethereum],[Maker],makerdao,8.521872e+09,1.048751e+09,9.400086e+08
1,240,Polygon Bridge & Staking,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,MATIC,Polygon,matic-network,0,3890,Chain,[Polygon],NaN,polygon-bridge-&-staking,7.423650e+09,9.255031e+09,7.410322e+09
2,182,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,Multi-Chain,lido-dao,2,8000,Liquid Staking,"[Ethereum, Solana, Moonriver, Terra]",NaN,lido,6.760504e+09,1.617410e+09,8.064785e+08
3,1,Uniswap,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,Multi-Chain,uniswap,2,7083,Dexes,"[Ethereum, Polygon, Arbitrum, Optimism, Celo]",[],uniswap,5.925136e+09,7.577257e+09,3.458939e+09
4,3,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,Multi-Chain,curve-dao-token,2,6538,Dexes,"[Ethereum, Polygon, Avalanche, Fantom, Arbitru...",[],curve,5.850475e+09,4.177814e+09,4.957653e+08
5,2,WBTC,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,Ethereum,wrapped-bitcoin,2,3717,Bridge,[Ethereum],[],wbtc,5.257923e+09,5.275027e+09,5.275027e+09
6,111,AAVE V2,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,Multi-Chain,aave,2,7278,Lending,"[Ethereum, Polygon, Avalanche]",[Chainlink],aave-v2,4.962335e+09,1.587153e+09,1.383680e+09
7,319,Convex Finance,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,CVX,Ethereum,convex-finance,2,9903,Yield,[Ethereum],NaN,convex-finance,3.909050e+09,7.401694e+08,4.851044e+08
8,494,JustLend,tron:TCFLL5dx5ZJdKnWuesXxi1VPwjLVmWZZy9,JST,Tron,just,2,5488,Lending,[Tron],[WINkLink],justlend,3.325727e+09,3.062259e+08,2.753584e+08
9,194,PancakeSwap,bsc:0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82,CAKE,Binance,pancakeswap-token,2,1165,Dexes,[Binance],[],pancakeswap,2.995309e+09,2.582471e+09,4.980615e+08


In [96]:
pdf_tvl_protocols[pdf_tvl_protocols['symbol'].str.startswith('F')]  

,id,name,address,symbol,chain,gecko_id,audits,cmcId,category,chains,oracles,slug,tvl,fdv,mcap
16,359,Frax,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,FXS,Multi-Chain,frax-share,2,6953,Algo-Stables,"[Ethereum, Fantom]",[Chainlink],frax,1.264818e+09,6.194929e+08,444817607.0
115,1882,Ferro,cronos:0x39bC1e38c842C60775Ce37566D03B41A7A66C782,FER,Cronos,ferro,2,20716,Dexes,[Cronos],[],ferro,9.555314e+07,5.279900e+08,74243266.0
149,255,Fei Protocol,0x956f47f50a910163d8bf957cf5846d573e7f87ca,FEI,Ethereum,fei-protocol,2,8642,Algo-Stables,[Ethereum],[Chainlink],fei-protocol,6.008664e+07,5.249708e+08,521800599.0
159,243,Flamincome,0x57dD84042ec9507963016596A34EdaD42F7e4CCE,FLAG,Ethereum,None,2,None,Yield Aggregator,[Ethereum],NaN,flamincome,5.477708e+07,NaN,NaN
164,265,Reflexer,0x6243d8cea23066d098a15582d81a598b4e8391f4,FLX,Ethereum,reflexer-ungovernance-token,2,9493,Algo-Stables,[Ethereum],[Chainlink],reflexer,5.240560e+07,3.853215e+07,19170484.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,946,FreeLiquid,eth:0xffed56a180f23fd32bc6a1d8d3c09c283ab594a8,FL,Ethereum,freeliquid,2,8437,Lending,[Ethereum],NaN,freeliquid,0.000000e+00,1.033019e+04,0.0
1589,1199,Ferrum,0xe5caef4af8780e59df925470b050fb23c43ca68c,FRM,Multi-Chain,ferrum-network,0,4228,Services,"[Ethereum, Polygon]",NaN,ferrum,0.000000e+00,8.330805e+06,2901463.0
1595,1219,Trapeza Protocol,bsc:0x414a36a4b79Ee1D4b454AB798E4179ffC00b1641,FIDL,Binance,trapeza-protocol,0,17625,Reserve Currency,[Binance],NaN,trapeza-protocol,0.000000e+00,NaN,0.0
1655,1446,FDoge Finance,fantom:0xEb0a2D1b1a33D95204af5d00f65FD9e349419878,FDOGE,Fantom,fdoge-finance,0,18314,Algo-Stables,[Fantom],[],fdoge-finance,0.000000e+00,NaN,0.0
